# Init

In [ ]:
# # if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget
# !pip install gensim
# !pip install catboost
# !pip install cython

In [ ]:
from __future__ import print_function
import os
import sys
import logging
import gc
import wget
import time
import tarfile
import zipfile
import functools
import random
import copy
import pickle
from tqdm import tqdm_notebook, tqdm
from itertools import product, combinations
from scipy.special import comb, perm

import scipy
import numpy as np
import pandas as pd
import catboost as cbt
from glove import *

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.linear_model import LogisticRegression,BayesianRidge,SGDClassifier,PassiveAggressiveClassifier,RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC,NuSVC,SVC
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Dropout, Activation, Conv1D, LSTM, GRU  #, CuDNNGRU, CuDNNLSTM
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import concatenate
from keras.callbacks import *
from keras.optimizers import *
from keras.engine.topology import Layer
from keras.layers import *
from keras.utils import np_utils

import gensim
import xgboost as xgb
import lightgbm as lgb

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# import ray
# ray.init(object_store_memory=int(100e6))
# import modin.pandas as pd
# os.environ["MODIN_ENGINE"] = "ray"  # Modin will use Ray
# os.environ["MODIN_ENGINE"] = "dask"  # Modin will use Dask

In [ ]:
class Logger(object):
    def __init__(self, filename='default.log', stream=sys.stdout):
        self.terminal = stream
        self.log = open(filename, 'a')
        
    def write(self, message):
        self.terminal.write(message)
        self.log.write(message)
        
    def flush(self):
        pass

In [ ]:
# sys.stdout = Logger("logs/default.log", sys.stdout)
# sys.stderr = Logger("logs/default_err.log", sys.stderr)

In [ ]:
DDIR = "data"
UDDIR = "user_data"
UFEDIR = "user_data/feat_data_v05"
UMDIR = "user_data/model_data"
RESDIR = "prediction_result"

In [ ]:
UID = "user_id"

# Load data (Only once)

In [ ]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [ ]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [ ]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [ ]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

In [ ]:
def mynunique(values):
    return values.nunique(dropna=False)
def getidxmax(x):
    return x.idxmax()[1]
# for time series
def at_len(x):
    return len(x)

def at_sum(x):
    return np.sum(x)

def at_max(x):
    return np.max(x)

def at_min(x):
    return np.min(x)

def at_mean(x):
    return np.mean(x)

def at_range(x):
    return at_max(x) - at_min(x)

def at_nunq(x):
    return len(set(x))

def at_lenDrange(x):
    return at_len(x)/(at_range(x)+1)

def at_lenDnunq(x):
    return at_len(x)/at_nunq(x)

def at_percentile(n):
    def at_percentile_(x):
        return np.percentile(x, n)
    at_percentile_.__name__ = f"at_percentile_{n}"
    return at_percentile_

In [ ]:
OP_SET = ["sum", "max", "min", "mean", "std"]
OP_SET1 = ["nunique", "sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]
OP_SET2 = ["sum", "max", "min", "mean", "std", "median", "skew", at_percentile(0.25), at_percentile(0.75)]

In [ ]:
ID_SET = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry", "time", "click_times"]

In [ ]:
nesting_level = 0
is_start = None
class Timer:
    def __init__(self):
        self.start = time.time()
        self.history = [self.start]

    def check(self, info):
        current = time.time()
        print(f"[{info}] spend {current - self.history[-1]:0.2f} sec")
        self.history.append(current)

def log(entry):
    global nesting_level
    space = "-" * (4 * nesting_level)
    print(f"{space}{entry}")

def timeit(method, start_log=None):
    @functools.wraps(method)
    def timed(*args, **kw):
        global is_start
        global nesting_level

        if not is_start:
            print()

        is_start = True
        log(f"Start [{method.__name__}]:" + (start_log if start_log else ""))
        log(f'Start time: {time.strftime("%Y-%m-%d %H:%M:%S")}')
        nesting_level += 1

        start_time = time.time()
        result = method(*args, **kw)
        end_time = time.time()

        nesting_level -= 1
        log(f"End   [{method.__name__}]. Time elapsed: {end_time - start_time:0.2f} sec.")
        is_start = False

        return result

    return timed

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Meta Train 

__weak learner prediction as features__

In [ ]:
# read train data
train_click_log = pd.read_csv(f"{DDIR}/train_preliminary/click_log.csv")
train_ad = pd.read_csv(f"{DDIR}/train_preliminary/ad.csv")
# tag
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [ ]:
# read test data
test_click_log = pd.read_csv(f"{DDIR}/test/click_log.csv")
test_ad = pd.read_csv(f"{DDIR}/test/ad.csv")
# pd.DataFrame(np.sort(test_click_log[UID].unique()), columns=[UID]).to_csv(f"{DDIR}/test/user.csv", index=False)
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

In [ ]:
train_click_log.shape

In [ ]:
train_ad.shape

In [ ]:
train_user.shape

In [ ]:
test_click_log.shape

In [ ]:
test_ad.shape

In [ ]:
train_ad["product_id"] = train_ad["product_id"].replace("\\N", -1).astype(int)
train_ad["industry"] = train_ad["industry"].replace("\\N", -1).astype(int)

In [ ]:
test_ad["product_id"] = test_ad["product_id"].replace("\\N", -1).astype(int)
test_ad["industry"] = test_ad["industry"].replace("\\N", -1).astype(int)

In [ ]:
# creative id in train (creative id is unique in train_ad)
len(train_ad)

In [ ]:
# creative id in test (creative id is unique in test_ad)
len(test_ad)

In [ ]:
tol_train = pd.merge(train_click_log, train_ad, how="left", on="creative_id")
tol_test = pd.merge(test_click_log, test_ad, how="left", on="creative_id")

In [ ]:
del train_click_log, train_ad
del test_click_log, test_ad

In [ ]:
#================Once=================#

In [ ]:
tol_test.index = -tol_test.index -1

In [ ]:
tol_data = pd.concat([tol_train, tol_test])

In [ ]:
del tol_train, tol_test
gc.collect()

In [ ]:
tol_data[f"{UID}_count"] = tol_data[[UID]].groupby([UID])[UID].transform("count")

In [ ]:
for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    tmp = tol_data[[UID, col]]
    tol_data[f"{UID}_{col}_std"] = tmp.groupby([UID])[col].transform("std")
    tol_data[f"{col}_{UID}_count"] = tmp.groupby([col])[UID].transform("count")
    tol_data[f"{col}_{UID}_std"] = tmp.groupby([col])[UID].transform("std")
    tmp = None

In [ ]:
for col in ["time", "click_times"]:
    tmp = tol_data[[UID, col]]
    tol_data[f"{UID}_{col}_std"] = tmp.groupby([UID])[col].transform("std")
    tol_data[f"{UID}_{col}_mean"] = tmp.groupby([UID])[col].transform("mean")
    tmp = None

In [ ]:
tol_data = reduce_mem_usage(tol_data)

In [ ]:
tol_data.to_pickle(f"{UDDIR}/imd/tol_data.pkl")

In [ ]:
#==============End===============#

In [ ]:
## if have tol_data.pkl
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")
test_user = pd.read_csv(f"{DDIR}/test/user.csv")

tol_data = pd.read_pickle(f"{UDDIR}/imd/tol_data.pkl")

In [ ]:
def rmse(y_true, y_pred):
    return (mean_squared_error(y_true, y_pred))** .5

In [ ]:
gc.collect()

## For age

In [ ]:
tol_data = pd.merge(tol_data, train_user[[UID, "age"]], how="left", on=UID)

In [ ]:
# use df_hist_train df_new_train df_hist_new_train to train 3 models
train_df = tol_data[tol_data["age"].notnull()]
test_df = tol_data[tol_data["age"].isnull()]

In [ ]:
drop_features = [UID, "age"]
cat_features = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
feats = [f for f in tol_data.columns if f not in drop_features]

In [ ]:
n_splits= 3
folds = GroupKFold(n_splits=n_splits)
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
print ('feats:' + str(len(feats)))

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df["age"],groups=train_df[UID])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df["age"].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df["age"].iloc[valid_idx] 
    
    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
               "objective" : "regression", 
               "boosting" : "gbdt", 
               "metric" : "rmse",  
               "max_depth": 7, 
               "num_leaves" : 31, 
               "max_bin" : 255, 
               "learning_rate" : 0.1, 
               "subsample" : 0.8,
               "colsample_bytree" : 0.8, 
               "verbosity": -1,
               "num_threads" : -1,
    }
    

    if n_fold >= 0:
        evals_result = {}
        dtrain = lgb.Dataset(
            train_x, label=train_y,categorical_feature=cat_features)
        dval = lgb.Dataset(
            valid_x, label=valid_y, reference=dtrain,categorical_feature=cat_features)
        bst = lgb.train(
            params, dtrain, num_boost_round=30000,
            valid_sets=[dval], early_stopping_rounds=100, verbose_eval=20,)#feval = evalerror
        
        new_list = sorted(zip(feats, bst.feature_importance('gain')),key=lambda x: x[1], reverse=True)[:]
        for item in new_list:
            print (item) 

        oof_preds[valid_idx] = bst.predict(valid_x, num_iteration=bst.best_iteration)

        sub_preds += bst.predict(test_df[feats], num_iteration=bst.best_iteration) / folds.n_splits # test_df_new

In [ ]:
cv = rmse(train_df["age"],  oof_preds)
print('Full OOF RMSE %.6f' % cv)  

In [ ]:
a = train_df[[UID]]
b = test_df[[UID]]

a["age_pred"] = oof_preds
b["age_pred"] = sub_preds

a1 = a.groupby([UID])["age_pred"].agg(["mean", "std", "min", "max", "median", "skew"])
b1 = b.groupby([UID])["age_pred"].agg(["mean", "std", "min", "max", "median", "skew"])

In [ ]:
a1.append(b1).add_prefix("age_gkf_agg_pred_").reset_index().sort_values(by=[UID]).reset_index(drop=True).to_pickle(f"{UFEDIR}/meta_age_group_regeress.pkl")

## For gender

In [ ]:
tol_data = pd.merge(tol_data, train_user[[UID, "gender"]], how="left", on=UID)

In [ ]:
# use df_hist_train df_new_train df_hist_new_train to train 3 models
train_df = tol_data[tol_data["gender"].notnull()]
test_df = tol_data[tol_data["gender"].isnull()]

In [ ]:
drop_features = [UID, "gender"]
cat_features = ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]

In [ ]:
feats = [f for f in tol_data.columns if f not in drop_features]

In [ ]:
n_splits= 3
folds = GroupKFold(n_splits=n_splits)
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
print ('feats:' + str(len(feats)))

for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df["gender"],groups=train_df[UID])):
    train_x, train_y = train_df[feats].iloc[train_idx], train_df["gender"].iloc[train_idx]
    valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df["gender"].iloc[valid_idx] 
    
    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
               "objective" : "regression", 
               "boosting" : "gbdt", 
               "metric" : "rmse",  
               "max_depth": 7, 
               "num_leaves" : 31, 
               "max_bin" : 255, 
               "learning_rate" : 0.1, 
               "subsample" : 0.8,
               "colsample_bytree" : 0.8, 
               "verbosity": -1,
               "num_threads" : -1,
    }
    

    if n_fold >= 0:
        evals_result = {}
        dtrain = lgb.Dataset(
            train_x, label=train_y,categorical_feature=cat_features)
        dval = lgb.Dataset(
            valid_x, label=valid_y, reference=dtrain,categorical_feature=cat_features)
        bst = lgb.train(
            params, dtrain, num_boost_round=30000,
            valid_sets=[dval], early_stopping_rounds=100, verbose_eval=20,)#feval = evalerror
        
        new_list = sorted(zip(feats, bst.feature_importance('gain')),key=lambda x: x[1], reverse=True)[:]
        for item in new_list:
            print (item) 

        oof_preds[valid_idx] = bst.predict(valid_x, num_iteration=bst.best_iteration)

        sub_preds += bst.predict(test_df[feats], num_iteration=bst.best_iteration) / folds.n_splits # test_df_new

In [ ]:
cv = rmse(train_df["gender"],  oof_preds)
print('Full OOF RMSE %.6f' % cv)

In [ ]:
a = train_df[[UID]]
b = test_df[[UID]]

a["gender_pred"] = oof_preds
b["gender_pred"] = sub_preds

a1 = a.groupby([UID])["gender_pred"].agg(["mean", "std", "min", "max", "median", "skew"])
b1 = b.groupby([UID])["gender_pred"].agg(["mean", "std", "min", "max", "median", "skew"])

In [ ]:
a1.append(b1).add_prefix("gender_gkf_agg_pred_").reset_index().sort_values(by=[UID]).reset_index(drop=True).to_pickle(f"{UFEDIR}/meta_gender_group_regeress.pkl")